## Add tool calling to engine

When I copied the engine code in challenge 20 I left out tool calling. When looking at the sft train run in challenge 30 (`challenge-30-sft-train-d20/look-into-observations.ipynb`) I realized that the chat eval I ran after the mid training and after the sft training for SpellingBee was not valid because the engine should have been using the python calculator.

First create a prompt that should make the assistant want to write python and generate with the current engine wihtout tool calling.

In [1]:
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_checkpoint_manager import load_model
from my_nanochat.my_common import compute_init, autodetect_device_type
from my_nanochat.my_engine import Engine
device_type = autodetect_device_type() 
_, _, _, _, device = compute_init(device_type)
model, tokenizer, meta_data = load_model('mid', model_tag='d20', device=device, phase='eval')
engine = Engine(model, tokenizer)

Autodetected device type: mps
loading the model from /Users/ericsilberstein/.cache/my_nanochat/mid_checkpoints/d20 with step 809
Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 20, 'n_head': 10, 'n_kv_head': 10, 'n_embd': 1280}


In [2]:
assistant_start = tokenizer.encode_special("<|assistant_start|>")
def get_conversation_start_tokens(user_content):
    tokens, _ = tokenizer.render_conversation(
        {
            'messages': [{'role': 'user', 'content': user_content}]
        })
    tokens.append(assistant_start)
    return tokens

In [3]:
tokens = get_conversation_start_tokens("How many e's are in yesterday?")

In [4]:
tokenizer.decode(tokens)

"<|bos|><|user_start|>How many e's are in yesterday?<|user_end|><|assistant_start|>"

In [5]:
results, mask = engine.generate_batch(tokens, max_tokens=512)

In [7]:
print(tokenizer.decode(results[0]))

<|bos|><|user_start|>How many e's are in yesterday?<|user_end|><|assistant_start|>We are asked to find the number 'e' in the word 'theotol'. Let me try a manual approach first.

First spell the word out:
theotol:t,h,e,o,t,o,l

Then count the occurrences of 'e':
1:t
2:h
3:e hit! count=1
4:o
5:t
6:o
7:l

This gives us 1.

Let me double check this using Python:

<|python_start|>'theotol'.count('e')<|python_end|><|output_start|>1<|output_end|>

Python gives us 1.

My final answer is:

#### 1<|assistant_end|>


^ wrong in many ways, although its prediction of what the python program will return is right

What if we format exactly like one of the templates in SpellingBee?

In [9]:
tokens = get_conversation_start_tokens("How many 'e' are in the word 'yesterday'?")
results, mask = engine.generate_batch(tokens, max_tokens=512)
print(tokenizer.decode(results[0]))

<|bos|><|user_start|>How many 'e' are in the word 'yesterday'?<|user_end|><|assistant_start|>We are asked to find the number 'e' in the word 'yesterday'. Let me try a manual approach first.

First spell the word out:
yesterday:y,member,s,o,o,n,y,e,r

Then count the occurrences of 'e':
1:y
2:member
3:s
4:o
5:o
6:n
7:y
8:e hit! count=1
9:r

This gives us 1.

Let me double check this using Python:

<|python_start|>'yesterday'.count('e')<|python_end|><|output_start|>1<|output_end|>

Python gives us 1.

My final answer is:

#### 1<|assistant_end|>


^ Interesting. It spells it out wrong, writes the python program right, but predicts the results of the python program wrong.

In [10]:
tokens = get_conversation_start_tokens("How many 'e' are in the word 'ever'?")
results, mask = engine.generate_batch(tokens, max_tokens=512)
print(tokenizer.decode(results[0]))

<|bos|><|user_start|>How many 'e' are in the word 'ever'?<|user_end|><|assistant_start|>We are asked to find the number 'e' in the word 'ever'. Let me try a manual approach first.

First spell the word out:
ever:e,v,e,r

Then count the occurrences of 'e':
1:e hit! count=1
2:v
3:e hit! count=2
4:r

This gives us 2.

Let me double check this using Python:

<|python_start|>'ever'.count('e')<|python_end|><|output_start|>2<|output_end|>

Python gives us 2.

My final answer is:

#### 2<|assistant_end|>


^ This one it does everything right.

### Add tool calling

Before I look at the code, think about how this is supposed to work.

- The model is generating tokens moving along 1 input and 1 output at a time
- The token <|python_start|> comes out
- "On the side", record the next token(s) up to but not including when <|python_end|> comes out
- Decode that, ask python to evaluate it, and capture the output
- Encode the output
- Append <|output_start|> tokens_of_encoded_output <|output_end|> to our result (or yield them)
- And also feed them as the a single multi-token input (this is the case where we get to use our "rectangle followed by triangle" mask as input to scaled dot product attention)
- Now the model generates a single token, whatever comes after <|output_end|>

ok, now start copying, start with the calculator stuff. Why can't / don't we use `execution.py` used to judge HumanEval?

#### calculator

In [1]:
import sys
sys.path.append('../my_nanochat')
import time
from my_nanochat.my_engine import timeout, eval_with_timeout, use_calculator

In [2]:
with timeout(1, 'foo'):
    print('foo')
    print('bar')

foo
bar


In [3]:
with timeout(1, 'foo'):
    print('foo')
    time.sleep(2)
    print('bar')

foo


Exception: 'foo' timed out after 1 seconds

In [4]:
eval_with_timeout('2+3')

5

In [5]:
# expect silent "fail"
eval_with_timeout('2+a')

In copying, realizing that my accuracy eval of others tasks (at least GSM8K) was also wrong because some (or most?) of those use math expressions.

In [6]:
use_calculator('2+3')

5

In [7]:
use_calculator('2,1+3')

24

In [9]:
use_calculator('"foo".count("o")')

2

In [15]:
use_calculator('os.system("ls")')

#### state machine

In copying code, realizing that how it actually works is similar to what I thought above but not exactly. After the initial prompt, we only ever input one token at a time (per row in batch) to the model. We need to do it this way so we can process in batches. (Say we're asking for 2 samples. If the first one is at a place where we now have 5 forced tokens to input to the model but the second has a single regular token we're stuck. We could potentially right pad the row with the 1 token and remember where to get our next token prediction from, but that's more confusing, and probably more wasteful.)

For now put a debug print statement in the engine before it calls the calculator to easily see if it's being called or not

In [1]:
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_checkpoint_manager import load_model
from my_nanochat.my_common import compute_init, autodetect_device_type
from my_nanochat.my_engine import Engine
device_type = autodetect_device_type() 
_, _, _, _, device = compute_init(device_type)
model, tokenizer, meta_data = load_model('mid', model_tag='d20', device=device, phase='eval')
engine = Engine(model, tokenizer)
assistant_start = tokenizer.encode_special("<|assistant_start|>")
def get_conversation_start_tokens(user_content):
    tokens, _ = tokenizer.render_conversation(
        {
            'messages': [{'role': 'user', 'content': user_content}]
        })
    tokens.append(assistant_start)
    return tokens

Autodetected device type: mps
loading the model from /Users/ericsilberstein/.cache/my_nanochat/mid_checkpoints/d20 with step 809
Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 20, 'n_head': 10, 'n_kv_head': 10, 'n_embd': 1280}


In [2]:
tokens = get_conversation_start_tokens("How many 'e' are in the word 'yesterday'?")
results, mask = engine.generate_batch(tokens, max_tokens=512)
print(tokenizer.decode(results[0]))

about to call use_calculator on 'yesterday'.count('e')
<|bos|><|user_start|>How many 'e' are in the word 'yesterday'?<|user_end|><|assistant_start|>We are asked to find the number 'e' in the word 'yesterday'. Let me try a manual approach first.

First spell the word out:
yesterday:y,member,s,o,o,n,y,e,r

Then count the occurrences of 'e':
1:y
2:member
3:s
4:o
5:o
6:n
7:y
8:e hit! count=1
9:r

This gives us 1.

Let me double check this using Python:

<|python_start|>'yesterday'.count('e')<|python_end|><|output_start|>2<|output_end|>

Python gives us 2.

My final answer is:

#### 2


^ worked exactly as expected / hoped ... exactly the same as above until the python output, but then we got the right output, it correctly said "Python gives us 2" and it correctly "used" that for its final answer. Also can see that the calculator was called.

Check the mask. Should see 3 forced tokens.

In [7]:
''.join(str(n) for n in mask[0])

'000000000000000000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110001111111111111111111'

^ right, a bunch for the user part in the beginning and then the 3 for the output

In [8]:
tokens = get_conversation_start_tokens("How many 'e' are in the word 'ever'?")
results, mask = engine.generate_batch(tokens, max_tokens=512)
print(tokenizer.decode(results[0]))

about to call use_calculator on 'ever'.count('e')
<|bos|><|user_start|>How many 'e' are in the word 'ever'?<|user_end|><|assistant_start|>We are asked to find the number 'e' in the word 'ever'. Let me try a manual approach first.

First spell the word out:
ever:e,v,e,r

Then count the occurrences of 'e':
1:e hit! count=1
2:v
3:e hit! count=2
4:r

This gives us 2.

Let me double check this using Python:

<|python_start|>'ever'.count('e')<|python_end|><|output_start|>2<|output_end|>

Python gives us 2.

My final answer is:

#### 2


In [9]:
tokens = get_conversation_start_tokens("If John has 2 trucks and Mary has 3 trucks, how many trucks do they have together?")
results, mask = engine.generate_batch(tokens, max_tokens=512)
print(tokenizer.decode(results[0]))

<|bos|><|user_start|>If John has 2 trucks and Mary has 3 trucks, how many trucks do they have together?<|user_end|><|assistant_start|>Let's denote the number of trucks John has as 't' and Mary's as 'm'. We are given the relationship 't <= m', so the equation becomes 't <= m * m'.

Since John has 2 trucks, we have 2m trucks. And we know that 'm * m' means there are 2 trucks. So, we can write 'm * m' = t.

Substituting 't' into the equation, we get 2 * (t <= m * m). We are given 'm * m' as the relationship, so we can simplify the equation to '2 * t ≤ m * m'.

Now, we can solve for 't'. Since t is an unknown, we will use the relationship between 'm', 't', and 'm' to do the math. Alright, we find that 'm' is 3. Then 't' is 2.

We can use our relationship between 'm', 't', and 'm' to find 't' when t = 2. So, when t = 2, we get 2 * (2 * 3) / 4 ≤ m * (2 * 3) / 4.

This simplifies to 2 * 2 * 3 * (t * (2 * 3) / 4). When t = 2, we have 2 * (2 * 4) / 4 and 2 * (2 * 2) / 4. These are both equal to

I could look for some of the < 4% of GSM8K problems the sft model got right (even without calculator use) and try one of those but will be slow to find on my mac. Hard to believe they are much easier than the one I made up though.

Try an actual one from the training set:

In [10]:
tokens = get_conversation_start_tokens("Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?")
results, mask = engine.generate_batch(tokens, max_tokens=512)
print(tokenizer.decode(results[0]))

<|bos|><|user_start|>Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?<|user_end|><|assistant_start|>Let S be the number of shells Kyle found.
1/3 is S=3
1/3*2 is S=6
A=12
The answer is: 12


^ Wrong answer and not using calculator.

Sanity check

In [11]:
tokens = get_conversation_start_tokens("Who are you?")
results, mask = engine.generate_batch(tokens, max_tokens=512)
print(tokenizer.decode(results[0]))

<|bos|><|user_start|>Who are you?<|user_end|><|assistant_start|>I am nanochat, an Large Language Model. I was built by King Andrej Karpathy in 2025. I am based on the Transformer neural network architecture.


^ in some earlier challenge when I tried that it kept going with a whole back and forth between user and assistant and that's because I only just added assistant_end as a stop condition in the generator

In [12]:
tokens = get_conversation_start_tokens("Who are you?")
results, mask = engine.generate_batch(tokens, num_samples=2, top_k=50, temperature=0.8, max_tokens=512)
print(tokenizer.decode(results[0]))
print()
print(tokenizer.decode(results[1]))

<|bos|><|user_start|>Who are you?<|user_end|><|assistant_start|>That's a great question! I'm nanochat, a Large Language Model. I was built by King Andrej Karpathy in 2025. You can find all my code on GitHub at https://github.com/karpathy/nanochat. It's all MIT licensed, so anyone can explore and hack on me!

<|bos|><|user_start|>Who are you?<|user_end|><|assistant_start|>That's me. I'm a digital AI assistant, and I'm here to assist you with programming and problem-solving tasks. I'm designed to help with a wide range of programming languages, including but not limited to Python, Java, JavaScript, and more. I can assist with debugging, troubleshooting, and optimizing code. What programming challenge or problem would you like help with?


#### Check chat_eval runs

In [14]:
import os
os.environ["PYTHONPATH"] = "../my_nanochat"
!python -m scripts.my_chat_eval --source=mid --batch-size=1 --model-tag=d4 --max-problems=5

Autodetected device type: mps
loading the model from /Users/ericsilberstein/.cache/my_nanochat/mid_checkpoints/d4 with step 9
Building model with config: {'sequence_len': 128, 'vocab_size': 65536, 'n_layer': 4, 'n_head': 2, 'n_kv_head': 2, 'n_embd': 256}
final: 0/5 (0.00%)
ARC-Easy accuracy: 0.00%
final: 0/5 (0.00%)
ARC-Challenge accuracy: 0.00%
final: 2/5 (40.00%)
MMLU accuracy: 40.00%
Rank 0 | 0/5 (0.00%)]
final: 0/5 (0.00%)
GSM8K accuracy: 0.00%
Rank 0 | 0/5 (0.00%)]
final: 0/5 (0.00%)
HumanEval accuracy: 0.00%
Rank 0 | 0/5 (0.00%)]
final: 0/5 (0.00%)
SpellingBee accuracy: 0.00%


Code added as part of this challenge

- Added calculator and updated row processing in generate() in `my_engine.py`